In [6]:
import sklearn
import xgboost
import pandas as pd
import scipy
import numpy as np
import _pickle as cPickle
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    auc,
    accuracy_score,
    confusion_matrix,
    mean_squared_error,
    r2_score,
)
from sklearn.model_selection import (
    cross_val_score,
    GridSearchCV,
    KFold,
    RandomizedSearchCV,
    train_test_split,
)
import xgboost as xgb
from scipy.stats import uniform, randint


In [7]:
with open(r"../Data/FBREF_Data.pickle", "rb") as inputfile:
    data = cPickle.load(inputfile)

with open(r"../Data/team.pickle", "rb") as inputfile:
    team = cPickle.load(inputfile)

In [8]:
data

,Player,Squad,Age,90s,SCA,SCA90,PassLive,PassDead,Drib,Sh,...,Def,GCA,GCA90,PassLive.1,PassDead.1,Drib.1,Sh.1,Fld.1,Def.1,-9999
0,Patrick van Aanholt,Crystal Palace,29,19.7,30,1.52,27,2,1,0,...,0,3,0.15,2,1,0,0,0,0,5f09991f
1,Tammy Abraham,Chelsea,22,11.6,17,1.47,9,0,2,4,...,0,2,0.17,1,0,0,0,1,0,f586779e
2,Che Adams,Southampton,24,29.6,68,2.29,50,0,2,7,...,0,7,0.24,6,0,0,0,1,0,f2bf1b0f
3,Tosin Adarabioyo,Fulham,22,32.8,18,0.55,18,0,0,0,...,0,1,0.03,1,0,0,0,0,0,c81d773d
4,Adrián,Liverpool,33,3.0,0,0.00,0,0,0,0,...,0,0,0.00,0,0,0,0,0,0,f76e6b4e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,Andi Zeqiri,Brighton,21,1.9,4,2.09,3,0,1,0,...,0,1,0.52,1,0,0,0,0,0,d01231f0
528,Oleksandr Zinchenko,Manchester City,23,16.4,32,1.95,26,4,0,1,...,0,2,0.12,2,0,0,0,0,0,51cf8561
529,Hakim Ziyech,Chelsea,27,13.0,73,5.60,54,7,4,7,...,0,6,0.46,5,1,0,0,0,0,6622454d
530,Kurt Zouma,Chelsea,25,22.5,14,0.62,11,0,0,3,...,0,0,0.00,0,0,0,0,0,0,ce4246f5


In [ ]:
numeric_features = df.T[6:].T.columns.to_list()
numeric_transformer = StandardScaler()

categorical_features = ['status']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

X = df[numeric_features+categorical_features]
y = df.total_points

preprocessor = ColumnTransformer(
    transformers = [
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

model = Pipeline(
    steps = [
        ('preprocessor', preprocessor),
        ('model', xgb.XGBRegressor())
    ]
)
X = df[numeric_features+categorical_features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [ ]:
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")



def display_scores(scores):
    print("Scores: {0}\nMean: {1:.3f}\nStd: {2:.3f}".format(scores, np.mean(scores), np.std(scores)))



In [ ]:
params = {
    'model__colsample_bytree' : uniform(0.7, 0.3),
    'model__gamma' : uniform(0, 0.5),
    'model__learning_rate' : uniform(0.03, 0.3),
    'model__max_depth' : randint(2, 6),
    'model__n_estimators' : randint(100, 150),
    'model__subsample' : uniform(0.6, 0.4)
}


search = RandomizedSearchCV(model, param_distributions=params, random_state=3791, n_iter=200, cv=3, verbose=1, return_train_score=True)

search.fit(X_train, y_train)

report_best_scores(search.cv_results_)


In [ ]:
y_pred = search.best_estimator_.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mse


In [ ]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmse

In [ ]:
r2 = r2_score(y_test, y_pred)
r2

In [ ]:
check = pd.DataFrame([y_pred, y_test], columns=y_test.index, index=['predict', 'test']).T

In [ ]:
check.sort_values(by='test', ascending=False)